<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2022_GoogleColabs/blob/main/Week2/AddGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 100         Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-01-10 17:02:27.867597510 +0000
Modify: 2022-01-10 17:02:27.758597720 +0000
Change: 2022-01-10 17:02:27.759597718 +0000
 Birth: -
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jcxkwsv7
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jcxkwsv7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=73fe2581dce991bfd7b8d45445f7ea9cac8b69e062359943d92d206743c29fc8
  Stored in directory: /tmp/pip-ephem-wheel-cache-s19gt9to/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [4]:
%%cu
#include <stdio.h>
#include <stdlib.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

//kernel function definition
__global__ void add(int *a , int *b, int *c) 
{
  *c = *a + *b;
}

int main() {
  int h_a = 10;
  int h_b = 20;
  int h_c = 10+20;

  int *d_a, *d_b, *d_c;

  //allocate memory on device
  cudaMalloc((void **)&d_a, sizeof(int));
  cudaMalloc((void **)&d_b, sizeof(int));
  cudaMalloc((void **)&d_c, sizeof(int));

  //copy host data to device memory
  cudaMemcpy(d_a, &h_a, sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &h_b, sizeof(int), cudaMemcpyHostToDevice);
  
  //call kernel
  add<<<1,1>>>(d_a, d_b, d_c); 

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");
  printf("Answer (on host): %d\n", h_c);

  //copy device data to host memory
  cudaMemcpy(&h_c, d_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("Answer (on device): %d", h_c);

  //release GPU memory
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  return 0;
}

Answer (on host): 30
Answer (on device): 30
